In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


with open('/Users/sofiawang/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [13]:
!pip install tqdm

In [14]:
with open('/Users/sofiawang/.secret/yelp_api.json') as f:   
    login = json.load(f)

yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [15]:
results = yelp_api.search_query(location='SF, CA',
                                       term='Thai')
print(type(results))
results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [16]:
LOCATION = 'SF,CA'
TERM = 'Thai'

In [17]:
JSON_FILE = "Data/results_in_progress_SF_thai.json"
JSON_FILE

'Data/results_in_progress_SF_thai.json'

In [18]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_SF_thai.json already exists.


In [19]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 1000 previous results found.


In [20]:
total_results= results['total']
total_results

1100

In [21]:
import pandas as pd

biz = pd.DataFrame(results['businesses'])
biz.head(2) 

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8kck3-K4zYKTJbJko0JlXQ,farmhouse-kitchen-thai-cuisine-san-francisco,Farmhouse Kitchen Thai Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/kUlEaP...,False,https://www.yelp.com/biz/farmhouse-kitchen-tha...,3719,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 37.760192883932795, 'longitude': ...","[delivery, restaurant_reservation, pickup]",$$,"{'address1': '710 Florida St', 'address2': '',...",+14158142920,(415) 814-2920,2201.588556
1,tZmiZw49SNRVzF6h1Jfx9g,ben-thai-cafe-san-francisco,Ben Thai Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/UVXRkn...,False,https://www.yelp.com/biz/ben-thai-cafe-san-fra...,1016,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.5,"{'latitude': 37.78924, 'longitude': -122.42065}","[delivery, pickup, restaurant_reservation]",$$,"{'address1': '1331 Polk St', 'address2': None,...",+14157712562,(415) 771-2562,3445.440156


In [22]:
results_per_page = len(results['businesses'])
results_per_page

20

In [23]:
import time, math

n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

5

In [24]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [25]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [26]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/5 [00:00<?, ?it/s]

In [27]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [28]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_SF_thai.json already exists. Deleting previous file...
[i] Data/results_in_progress_SF_thai.json not found. Saving empty list to new file.
- 0 previous results found.


55

In [29]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/55 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [30]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8kck3-K4zYKTJbJko0JlXQ,farmhouse-kitchen-thai-cuisine-san-francisco,Farmhouse Kitchen Thai Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/kUlEaP...,False,https://www.yelp.com/biz/farmhouse-kitchen-tha...,3719,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 37.760192883932795, 'longitude': ...","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '710 Florida St', 'address2': '',...",+14158142920,(415) 814-2920,2201.588556
1,tZmiZw49SNRVzF6h1Jfx9g,ben-thai-cafe-san-francisco,Ben Thai Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/UVXRkn...,False,https://www.yelp.com/biz/ben-thai-cafe-san-fra...,1016,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.5,"{'latitude': 37.78924, 'longitude': -122.42065}","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '1331 Polk St', 'address2': None,...",+14157712562,(415) 771-2562,3445.440156
2,8LFfuBSFK9YfHp1r-jsJEw,oraan-thai-eatery-san-francisco,Oraan Thai Eatery,https://s3-media2.fl.yelpcdn.com/bphoto/8buUkX...,False,https://www.yelp.com/biz/oraan-thai-eatery-san...,44,"[{'alias': 'thai', 'title': 'Thai'}]",4.5,"{'latitude': 37.78141, 'longitude': -122.459837}",[],NaN,"{'address1': '3750 Geary Blvd', 'address2': ''...",+16282197598,(628) 219-7598,3074.144773
3,AzFqspVE3gUWYaEselXLUA,khob-khun-thai-cuisine-and-breakfast-san-franc...,Khob Khun Thai Cuisine & Breakfast,https://s3-media3.fl.yelpcdn.com/bphoto/vmbbOH...,False,https://www.yelp.com/biz/khob-khun-thai-cuisin...,238,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 37.78107129493808, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '3741 Geary Blvd', 'address2': No...",+14153794382,(415) 379-4382,3032.307778
4,s2R3RUJ09eBFvHVIUS4Shg,kothai-republic-san-francisco,Kothai Republic,https://s3-media3.fl.yelpcdn.com/bphoto/eWlrib...,False,https://www.yelp.com/biz/kothai-republic-san-f...,247,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 37.762389, 'longitude': -122.466065}","[pickup, delivery]",NaN,"{'address1': '1398 9th Ave', 'address2': None,...",+14157424058,(415) 742-4058,2606.596929


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,5VAapc_o_KoQP9nAyh4iUQ,kirala-restaurant-berkeley-2,Kirala Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/qK3FBg...,False,https://www.yelp.com/biz/kirala-restaurant-ber...,1856,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 37.859286, 'longitude': -122.266717}","[pickup, delivery]",$$,"{'address1': '2100 Ward St', 'address2': '', '...",+15105493486,(510) 549-3486,18493.578677
996,qy7fp1aJXKdmWlOhlnlg4Q,fire-wings-oakland-oakland,Fire Wings Oakland,https://s3-media4.fl.yelpcdn.com/bphoto/4Rb5uR...,False,https://www.yelp.com/biz/fire-wings-oakland-oa...,85,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.0,"{'latitude': 37.799706245183614, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '203 E 18th St', 'address2': None...",+15108911148,(510) 891-1148,16627.166348
997,ouAw6oihgt3Ricwcbf1fMA,pho-anh-dao-alameda-alameda,Pho Anh Dao Alameda,https://s3-media2.fl.yelpcdn.com/bphoto/pk5SKY...,False,https://www.yelp.com/biz/pho-anh-dao-alameda-a...,162,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",3.5,"{'latitude': 37.778843, 'longitude': -122.276938}",[delivery],$$,"{'address1': '1919 Webster St', 'address2': ''...",+15102638577,(510) 263-8577,14161.625676
998,7GuQ9-aCQ7m-A-I-1tRAoQ,vietnam-house-alameda,Vietnam House,https://s3-media2.fl.yelpcdn.com/bphoto/nD6aOP...,False,https://www.yelp.com/biz/vietnam-house-alameda...,184,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",3.5,"{'latitude': 37.7661670327951, 'longitude': -1...",[delivery],$$,"{'address1': '1529 Park St', 'address2': '', '...",+15108644022,(510) 864-4022,17159.384216
999,p5TfqFI1lwcuhbI9DrXUTw,sushi-and-more-alameda,Sushi & More,https://s3-media1.fl.yelpcdn.com/bphoto/nGOfek...,False,https://www.yelp.com/biz/sushi-and-more-alamed...,168,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 37.76209, 'longitude': -122.24477}","[pickup, delivery]",$$,"{'address1': '1245 Park St', 'address2': None,...",+15102638171,(510) 263-8171,16845.624286


In [31]:
final_df.duplicated(subset='id').sum()

9

In [32]:
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [33]:
final_df.to_csv('Data/final_results_SF_thai.csv.gz', compression='gzip',index=False)